<h1>Computer Vision Problem</h1>

<h3>Instructions</h3>
<ul>
<li>This dataset has 4 classes. Labels for training data are provided, you have to submit labels of test data. Feel free to use any Machine learning or Deep learning technique.</li>
<li>In the form, you would have to submit the CSV file of test labels. CSV file should be named as your_full_name.csv each row of this file should be the following:- Test_image_index, predicted class</li>    
<li> Dataset folder has a sample submission file. Strictly follow that format(Follow the exact format. Make sure case and the name of headers are same. No extra index columns).Your GitHub repo should have code in Jupyter notebook.</li>    
</ul>
<hr />

<h3>Solution</h3>
<h4>I have used the following models for the CV task in the order listed below:</h4>
<ul>
    <li> Machine Learning Techniques </li>
    <ul>
        <li><b>KNN:</b> Used just for the sake of seeing the accuracy</li>
        <li></li>
        <li></li>
    </ul>
    <li> Deep Learning models </li>
</ul>

In [5]:
# Importing the necessary libraries for completing the task

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier as knn
import warnings 
%matplotlib inline

warnings.filterwarnings('ignore')

### Basic preprocessing on the dataset before any algorithm

In [ ]:
data = pd

#### Algorithm 1: K Nearest Neighbors